In [6]:
# 연습문제 : 청와대 국민청원 게시판 크롤링하기
# http://www1.president.go.kr/petitions

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy 
import pandas as pd  
import xlwt
import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *100)
print("연습문제 : 청와대 국민청원 게시판 크롤링하기")
print("=" *100)

query_txt = '청와대국민청원'

cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 7)
print("크롤링 할 총 페이지 번호: ",page_cnt)

f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir=='' :
    f_dir ='c:\\temp\\'

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_90/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('http://www1.president.go.kr/petitions')
driver.maximize_window()

#안내창 닫기
try :
    driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/table/tbody/tr/td[2]/div').click()
except :
    print('')
    
time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

print("\n")

#Step 4. 각 게시글의 상세 정보를 수집합니다

click_count = 1

# 각 게시글의 URL 주소를 추출하여 목록을 먼저 만든 후 상세 크롤링을 진행합니다.
# URL 수집하여 목록 만들기
url_list=[]
count = 0
#while ( click_count <= page_cnt + 1)  :
for a in range(1,page_cnt+1) :
    
    print("%s 번째 페이지에서 게시글의 URL 정보를 추출합니다==============" %click_count)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    result_1 = soup.find_all('div','board text')
    result_2= result_1[2].find('div', class_='b_list category b_list2') 
    result_url_list=result_2.find('div','bl_body').find('ul','petition_list').find_all('li')
        
    for i in result_url_list:
        url = i.find('a')['href']
        print(url)
        full_url = 'http://www1.president.go.kr/'+ url
        url_list.append(full_url)
            
        count += 1

        if count == cnt :
            break  

    print("%s 번째 페이지까지 %s건 정보 수집 완료 ====================" %(a,count))
    print("\n")
    
    a += 1
    try :
        driver.find_element_by_link_text("%s" %a).click()
    except :
        driver.find_element_by_link_text('Next').click()
        
    time.sleep(2)
        
# Step 5. 각 게시글 별 상세 내역을 수집합니다
no = 1

cno2=[]
title2=[]
people2=[]
category2=[]
s_date2=[]
e_date2=[]
content2=[]

for i in range(0,len(url_list)):  
            
    print("\n")
    print("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no)

    driver.get(url_list[i])

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    content_all = soup.find('div','petitionsView_left_pg')

    f = open(ff_name, 'a',encoding='UTF-8')

    # 청원 등록 번호
    c_no = url_list[i].split('/')
    #cno_1 = c_no[4].split('?')
    cno_1 = c_no[2]        
    print('1.청원글번호: ',cno_1 ,'\n')
    f.write('\n')
    f.write("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no + '\n')
    f.write('1.청원번호:' + cno_1 + '\n')
    cno2.append(cno_1)

    # 청원제목 추출
    title = content_all.find('h3','petitionsView_title').get_text()
    print('2.청원제목: ' , title , '\n')
    f.write('2.청원제목:' + title + '\n')
    title2.append(title)

    # 참여인원 추출
    people = content_all.find('h2','petitionsView_count').find('span','counter').get_text()
    print('3.참여인원: ' , people ,'명' ,'\n')
    f.write('3.참여인원:' + people + '\n')
    people2.append(people)

    # 카테고리
    cat_all = content_all.find('div','petitionsView_info').find('ul','petitionsView_info_list').find_all('li')

    category = cat_all[0].get_text().replace('카테고리','')
    print('4.카테고리: ',category ,'\n')
    f.write('4.카테고리:' + category + '\n')
    category2.append(category)

    # 청원시작일
    s_date = cat_all[1].get_text().replace('청원시작','')
    print('5.청원시작일: ',s_date ,'\n')
    f.write('5.청원시작일:' + s_date + '\n')
    s_date2.append(s_date)

    # 청원마감일
    e_date = cat_all[2].get_text().replace('청원마감','')
    print('6.청원마감일: ',e_date ,'\n')
    f.write('6.청원종료일:' + e_date + '\n')
    e_date2.append(e_date)

    # 청원내용
    content = content_all.find('div','View_write').get_text().replace('\n','')
    print('7.청원내용: ', content.strip() , '\n')
    f.write('7.청원내용:' + content.strip() + '\n')
    content2.append(content)

    f.close( )

    no += 1

    if no > cnt :
        break
            
# Step 6. 출력 결과를 저장하기
# 출력 결과를 표(데이터 프레임) 형태로 만들기

blue_house = pd.DataFrame()

blue_house['청원번호'] = cno2
blue_house['청원제목'] = pd.Series(title2) 
blue_house['참여자'] = pd.Series(people2)
blue_house['카테고리'] = pd.Series(category2)
blue_house['청원시작일'] = pd.Series(s_date2)
blue_house['청원종료일'] = pd.Series(e_date2)
blue_house['청원내용'] = pd.Series(content2)
       
# csv 형태로 저장하기
blue_house.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
blue_house.to_excel(fx_name , index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

#Step 8. 요약정보 출력하기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )

연습문제 : 청와대 국민청원 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 85
크롤링 할 총 페이지 번호:  13
2.결과 파일을 저장할 폴더명만 쓰세요(기본값:c:\temp\):


WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=91.0.4472.124)
